# EvolveGAT

### Dependency Import

In [ ]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/benedekrozemberczki/pytorch_geometric_temporal.git
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

     |████████████████████████████████| 7.9 MB 12.3 MB/s 
     |████████████████████████████████| 3.5 MB 12.7 MB/s 
     |████████████████████████████████| 407 kB 9.4 MB/s 


### Dataset

In [ ]:
import torch
from torch_geometric.loader import DataLoader 
import torch_geometric.transforms as T
import random

In [ ]:
#Bitcoin OTC
import torch
from torch_geometric.datasets import BitcoinOTC
from torch_geometric.loader import DataLoader 
import torch_geometric.transforms as T
import random

dataset = BitcoinOTC(root = './data')
dataset = dataset[:50]
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')
print(f'Is Undirected:  {dataset[0].is_undirected()}')

from sklearn.model_selection import train_test_split

#train_test split
#we can also use temporal_signal_split
train_dataset, test_dataset = train_test_split(dataset, test_size = 0.2, shuffle = False)
print(dataset[0])

print("")

Extracting data/raw/soc-sign-bitcoinotc.csv.gz
Processing...
Done!


Dataset: BitcoinOTC(50):
Number of graphs: 50
Number of features: 0
Number of classes: 0
Is Undirected:  False
Data(edge_index=[2, 41], edge_attr=[41], num_nodes=6005)



In [ ]:
print(len(train_dataset))
print(len(test_dataset))

40
10


### Helper Functions

In [ ]:
def normalizeAdjacency(W):
    # Check that the matrix is square
    assert W.shape[0] == W.shape[1]
    # Compute the degree vector
    d = torch.sum(W, axis = 1)
    # Invert the square root of the degree
    d = 1/torch.sqrt(d)
    # And build the square root inverse degree matrix
    D = torch.diag(d)
    # Return the Normalized Adjacency
    return D @ W @ D 

#Snapshot to adjacency matrix
def snap_to_adjmat(snapshot):
    x = torch.FloatTensor(torch.zeros(snapshot.num_nodes, snapshot.num_nodes))
    for j in range(snapshot.edge_index.size()[1]):
        v1 = snapshot.edge_index[:,j][0].item()
        v2 = snapshot.edge_index[:,j][1].item()
        x[v1][v2] = 1.0
    for i in range(snapshot.num_nodes):
        x[i][i] = 1.0

    #symetric normalisation(High time complexity)
    #x = normalizeAdjacency(x);
    return x

In [ ]:
temp = snap_to_adjmat(dataset[0])
print(temp.type())
print(temp)

torch.FloatTensor
tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])


In [ ]:
#Similarity index
def similarity(vec1, vec2):
    return torch.dot(vec1, vec2)

In [ ]:
from typing import Optional, Tuple

import torch
from torch import Tensor
from torch.nn import GRU
from torch_geometric.typing import Adj, OptTensor
from torch_sparse import SparseTensor
from torch_geometric.nn.inits import glorot
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.conv.gcn_conv import gcn_norm

class GCNConv_Fixed_W(MessagePassing):
    _cached_edge_index: Optional[Tuple[Tensor, Tensor]]
    _cached_adj_t: Optional[SparseTensor]

    def __init__(self, in_channels: int, out_channels: int,
                 improved: bool = False, cached: bool = False,
                 add_self_loops: bool = True, normalize: bool = True,
                **kwargs):

        kwargs.setdefault('aggr', 'add')
        super(GCNConv_Fixed_W, self).__init__(**kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.improved = improved
        self.cached = cached
        self.add_self_loops = add_self_loops
        self.normalize = normalize

        self._cached_edge_index = None
        self._cached_adj_t = None

        self.reset_parameters()

    def reset_parameters(self):
        self._cached_edge_index = None
        self._cached_adj_t = None


    def forward(self, W: torch.FloatTensor, x: Tensor, edge_index: Adj,
                edge_weight: OptTensor = None) -> Tensor:

        if self.normalize:
            cache = self._cached_edge_index
            if cache is None:
                edge_index, edge_weight = gcn_norm(  # yapf: disable
                    edge_index, edge_weight, x.size(self.node_dim),
                    self.improved, self.add_self_loops, dtype = float)

        x = torch.matmul(x, W)

        # propagate_type: (x: Tensor, edge_weight: OptTensor)
        out = self.propagate(edge_index, x=x, edge_weight=edge_weight,
                             size=None)

        return out


    def message(self, x_j: Tensor, edge_weight: OptTensor) -> Tensor:
        return x_j if edge_weight is None else edge_weight.view(-1, 1) * x_j

class attention(torch.nn.Module):
  def __init__(self):
    super(attention, self).__init__()

  def forward(self, X: Tensor, edge_index: Adj) -> Tensor:
    embed_prod = torch.zeros(X.shape[0], X.shape[0])
    for i in range(X.shape[0]):
      embed_prod[i] = torch.exp(torch.sum(torch.mul(X[i], X), axis = 1))
    
    alpha = torch.zeros(X.shape[0])
    edges = edge_index.T
    for edge in edges:
      alpha[edge[0]] += embed_prod[edge[0]][edge[1]]
    
    for i in range(X.shape[0]):
      alpha[i] += embed_prod[i][i]
    
    for i in range(X.shape[0]):
      embed_prod[i] /= alpha[i]
#    Y = X.detach()
    Y = torch.zeros(X.shape[0], X.shape[1]).to(device)
    for i in range(X.shape[0]):
      Y[i] = torch.mul(embed_prod[i][i], X[i])
    
    for edge in edges:
      Y[edge[0]] += torch.mul(embed_prod[edge[0]][edge[1]], X[edge[1]])
    return X
    
class EvolveGCNO_(torch.nn.Module):
    def __init__(
        self,
        in_channels: int,
        improved: bool = False,
        cached: bool = False,
        normalize: bool = True,
        add_self_loops: bool = True,
    ):
        super(EvolveGCNO_, self).__init__()

        self.in_channels = in_channels
        self.improved = improved
        self.cached = cached
        self.normalize = normalize
        self.add_self_loops = add_self_loops
        self.weight = None
        self.initial_weight = torch.nn.Parameter(torch.Tensor(in_channels, in_channels))
        self._create_layers()
        self.reset_parameters()
    
    def reset_parameters(self):
        glorot(self.initial_weight)


    def _create_layers(self):

        self.recurrent_layer = GRU(
            input_size=self.in_channels, hidden_size=self.in_channels, num_layers=1
        )
        for param in self.recurrent_layer.parameters():
            param.requires_grad = True
            param.retain_grad()

        self.conv_layer = GCNConv_Fixed_W(
            in_channels=self.in_channels,
            out_channels=self.in_channels,
            improved=self.improved,
            cached=self.cached,
            normalize=self.normalize,
            add_self_loops=self.add_self_loops
        )

        self.attention_layer = attention()

    def forward(
        self,
        X: torch.FloatTensor,
        edge_index: torch.LongTensor,
        edge_weight: torch.FloatTensor = None,
    ) -> torch.FloatTensor:

        if self.weight is None:
            self.weight = self.initial_weight.data
        W = self.weight[None, :, :]
        _, W = self.recurrent_layer(W, W)
        X = self.conv_layer(W.squeeze(dim=0), X, edge_index, edge_weight)
        X = self.attention_layer(X, edge_index)
        return X

### Model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
num_nodes = dataset[0].num_nodes

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import EvolveGCNO
from torch_geometric.nn import GATConv
import torch.nn as nn

class EvolveGAT(torch.nn.Module):
    def __init__(self, in_channels, emb_dim):
        super(EvolveGAT, self).__init__()
        self.linear = torch.nn.Linear(num_nodes, emb_dim)
        self.relu = nn.ReLU()
        self.recurrent = EvolveGCNO_(in_channels, add_self_loops=False, normalize=False)
        
    #forward propogation
    def encode(self, x, edge_index, edge_weight):
        h = self.linear(x)
        h = F.relu(h)
        h = self.recurrent(h, edge_index, edge_weight)
        return h

    #for edge classification (per edge)
    def decode(self, z, pos_edge_index, neg_edge_index): # only pos and neg edges
        edge_index = torch.cat([pos_edge_index, neg_edge_index], dim=-1) # concatenate pos and neg edges
        logits = (z[edge_index[0]] * z[edge_index[1]]).sum(dim=-1)  # dot product 
        return logits

    #for all edge 
    def decode_all(self, z): 
        prob_adj = z @ z.t() # get adj NxN
        return (prob_adj > 0).nonzero(as_tuple=False).t() # get predicted edge_list 


In [ ]:
print(dataset[0])
temp = train_dataset[0].edge_attr
print(temp)

Data(edge_index=[2, 41], edge_attr=[41], num_nodes=6005)
tensor([ 4,  2,  1,  7,  8,  8,  1,  5,  5,  5,  8,  8,  9,  7,  5,  1,  8,  7,
         8,  1, 10,  7,  7,  1,  1,  3,  3,  1,  4,  2,  5,  5,  1,  2,  2,  2,
         2,  2,  1,  2,  1])


### Data Management For Model

In [ ]:
from torch_geometric.utils import train_test_split_edges
from torch_geometric.utils import negative_sampling
import torch.nn.functional as F

In [ ]:
def get_link_labels(pos_edge_index, neg_edge_index):
    # returns a tensor:
    # [1,1,1,1,...,0,0,0,0,0,..] with the number of ones is equel to the lenght of pos_edge_index
    # and the number of zeros is equal to the length of neg_edge_index
    E = pos_edge_index.size(1) + neg_edge_index.size(1)
    link_labels = torch.zeros(E, dtype=torch.float, device=device)
    link_labels[:pos_edge_index.size(1)] = 1.
    return link_labels

In [ ]:
neg_edge_index = negative_sampling(
    edge_index = dataset[0].edge_index, #positive edges
    num_nodes = dataset[0].num_nodes, # number of nodes
    num_neg_samples = dataset[0].edge_index.size(1)
) 
pos_edge_index = dataset[0].edge_index

In [ ]:
#Check fns.
print(pos_edge_index.size())
print(neg_edge_index.size())
E = pos_edge_index.size(1) + neg_edge_index.size(1)
link_labels = torch.zeros(E, dtype=torch.float, device=device)
link_labels[:pos_edge_index.size(1)] = 1.
print(link_labels)

torch.Size([2, 41])
torch.Size([2, 41])
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')


### Model Train

In [ ]:
#Hyperparameters
emb_dim = 30

In [ ]:
from tqdm import tqdm
model = EvolveGAT(in_channels = emb_dim, emb_dim = emb_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

def train():
    for epoch in tqdm(range(40)):
        cost = 0
        for time, snapshot in enumerate(train_dataset):
            adj_mat = snap_to_adjmat(snapshot).to(device)
            next_snap = dataset[time]

            neg_edge_index = negative_sampling(
                edge_index = next_snap.edge_index, #positive edges
                num_nodes = next_snap.num_nodes, # number of nodes
                num_neg_samples = next_snap.edge_index.size(1)
                ) # number of neg_sample equal to number of pos_edges

            optimizer.zero_grad()

            y_hat = model.encode(adj_mat, snapshot.edge_index.to(device), edge_weight = None)
            link_logits = model.decode(y_hat, next_snap.edge_index, neg_edge_index) # decode
            link_labels = get_link_labels(next_snap.edge_index, neg_edge_index)
            
            loss = F.binary_cross_entropy_with_logits(link_logits, link_labels)
            loss.backward()
            cost += loss.item()
            optimizer.step()
        cost /= len(train_dataset) 
        print(f'Time: {epoch}', f'Cost: {cost}')
    return cost
#torch.autograd.set_detect_anomaly(True)
train()

  2%|▎         | 1/40 [01:40<1:05:30, 100.78s/it]

Time: 0 Cost: 0.4413000546395779


  5%|▌         | 2/40 [03:21<1:03:37, 100.47s/it]

Time: 1 Cost: 0.4511335365474224


  8%|▊         | 3/40 [05:00<1:01:42, 100.06s/it]

...

 98%|█████████▊| 39/40 [1:04:55<01:39, 99.13s/it]

Time: 38 Cost: 0.3705383911728859


100%|██████████| 40/40 [1:06:33<00:00, 99.84s/it]

Time: 39 Cost: 0.36832466796040536


0.36832466796040536

### Model Test

In [ ]:
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score

In [ ]:
model.eval()

EvolveGAT(
  (linear): Linear(in_features=6005, out_features=30, bias=True)
  (relu): ReLU()
  (recurrent): EvolveGCNO_(
    (recurrent_layer): GRU(30, 30)
    (conv_layer): GCNConv_Fixed_W(30, 30)
    (attention_layer): attention()
  )
)

In [ ]:
print('AUC Score')
@torch.no_grad()
def test():
    perf = []
    for time, snapshot in enumerate(test_dataset[:-1]):
        adj_mat = snap_to_adjmat(snapshot).to(device)
        next_snap = dataset[time]

        pos_edge_index = next_snap.edge_index
        neg_edge_index = negative_sampling(
            edge_index = next_snap.edge_index, #positive edges
            num_nodes = next_snap.num_nodes, # number of nodes
            num_neg_samples = next_snap.edge_index.size(1)
            ) # number of neg_sample equal to number of pos_edges


        y_hat = model.encode(adj_mat, snapshot.edge_index.to(device), edge_weight = None)
        link_logits = model.decode(y_hat, next_snap.edge_index, neg_edge_index) # decode
        link_probs = link_logits.sigmoid() # apply sigmoid
        link_labels = get_link_labels(next_snap.edge_index, neg_edge_index)
        
        auc = roc_auc_score(link_labels.cpu(), link_probs.cpu())
        print(f'Time: {time} -> AUC: {auc}')
        perf.append(roc_auc_score(link_labels.cpu(), link_probs.cpu())) #compute roc_auc score 
    return perf
perf = test()

AUC Score
Time: 0 -> AUC: 0.5627602617489589
Time: 1 -> AUC: 0.6180555555555556
Time: 2 -> AUC: 0.6039603960396039
Time: 3 -> AUC: 0.5948228634039444
Time: 4 -> AUC: 0.5705174927113703
Time: 5 -> AUC: 0.5533479321333831
Time: 6 -> AUC: 0.5600595151691101
Time: 7 -> AUC: 0.542219257448605
Time: 8 -> AUC: 0.5349086969093261


In [ ]:
print('Precision Score')
@torch.no_grad()
def test():
    perf = []
    for time, snapshot in enumerate(test_dataset[:-1]):
        adj_mat = snap_to_adjmat(snapshot).to(device)
        next_snap = dataset[time]

        pos_edge_index = next_snap.edge_index
        neg_edge_index = negative_sampling(
            edge_index = next_snap.edge_index, #positive edges
            num_nodes = next_snap.num_nodes, # number of nodes
            num_neg_samples = next_snap.edge_index.size(1)
            ) # number of neg_sample equal to number of pos_edges


        y_hat = model.encode(adj_mat, snapshot.edge_index.to(device), edge_weight = None)
        link_logits = model.decode(y_hat, next_snap.edge_index, neg_edge_index) # decode
        link_probs = link_logits.sigmoid() # apply sigmoid
        link_labels = get_link_labels(next_snap.edge_index, neg_edge_index)
        
        auc = precision_score(link_labels.cpu(), link_probs.cpu())
        print(f'Time: {time} -> AUC: {auc}')
        perf.append(roc_auc_score(link_labels.cpu(), link_probs.cpu())) #compute roc_auc score 
    return perf
perf = test()

In [ ]:
print('Recall Score')
@torch.no_grad()
def test():
    perf = []
    for time, snapshot in enumerate(test_dataset[:-1]):
        adj_mat = snap_to_adjmat(snapshot).to(device)
        next_snap = dataset[time]

        pos_edge_index = next_snap.edge_index
        neg_edge_index = negative_sampling(
            edge_index = next_snap.edge_index, #positive edges
            num_nodes = next_snap.num_nodes, # number of nodes
            num_neg_samples = next_snap.edge_index.size(1)
            ) # number of neg_sample equal to number of pos_edges


        y_hat = model.encode(adj_mat, snapshot.edge_index.to(device), edge_weight = None)
        link_logits = model.decode(y_hat, next_snap.edge_index, neg_edge_index) # decode
        link_probs = link_logits.sigmoid() # apply sigmoid
        link_labels = get_link_labels(next_snap.edge_index, neg_edge_index)
        
        auc = recall_score(link_labels.cpu(), link_probs.cpu())
        print(f'Time: {time} -> AUC: {auc}')
        perf.append(roc_auc_score(link_labels.cpu(), link_probs.cpu())) #compute roc_auc score 
    return perf
perf = test()

In [ ]:
print('F1 Score')
@torch.no_grad()
def test():
    perf = []
    for time, snapshot in enumerate(test_dataset[:-1]):
        adj_mat = snap_to_adjmat(snapshot).to(device)
        next_snap = dataset[time]

        pos_edge_index = next_snap.edge_index
        neg_edge_index = negative_sampling(
            edge_index = next_snap.edge_index, #positive edges
            num_nodes = next_snap.num_nodes, # number of nodes
            num_neg_samples = next_snap.edge_index.size(1)
            ) # number of neg_sample equal to number of pos_edges


        y_hat = model.encode(adj_mat, snapshot.edge_index.to(device), edge_weight = None)
        link_logits = model.decode(y_hat, next_snap.edge_index, neg_edge_index) # decode
        link_probs = link_logits.sigmoid() # apply sigmoid
        link_labels = get_link_labels(next_snap.edge_index, neg_edge_index)
        
        auc = recall_score(link_labels.cpu(), link_probs.cpu())
        print(f'Time: {time} -> AUC: {auc}')
        perf.append(roc_auc_score(link_labels.cpu(), link_probs.cpu())) #compute roc_auc score 
    return perf
perf = test()